# FEC Application Programming Interface (API) Module

In this tutorial, I will be walking you through how to use the Federal Election Commission's (FEC) API. APIs are powerful tools that many websites use for data storage and access. It will not be long before online data is primarily retrieved via APIs, so it is important to know the basic structure and skills required to do so. We will use the FEC API to first look up candidates in the Connecticut 2018 midterm election for the house of representatives to get a candidate ID given by the FEC. Then we will use a candidate ID to look up that candidate's history of campaign finance for as long as the representative has been in office. Finally, we will plot the total disbursements per campaign cycle.

## Module Outline
Step One: Get an API Key.<br>
Step Two: Documentation.<br>
Step Three: Initialize API Variables and Import Libraries<br>
Step Four: Candidate Search<br>
Step Five: Convert Data<br>
Step Six: Use Candidate ID to Retrieve Campaign Finance Data<br>
Step Seven: Visualize data<br>
Conclusion<br>

## Important Terminology
<b> Application Programming Interface (API):</b> Functions that allow for access to features or data of an operating system, applications, or online services. <br>
<b> Data Scraping:</b> The process of using an API to import or download data from a website into a local machine. <br>
<b> API Key:</b> APIs require an access key that behaves like a password to identify users who are accessing the data. <b> Do not</b> share your API key with anyone, nor make it accessible or visible in a public presentation or Jupyter Notebook. <br>

***

## Contact Information
Garrett Morrow <br>
Digital Teaching Integration Research Fellow <br>
PhD Student, Political Science <br>
morrow.g [at] husky [dot] neu [dot] edu
***

## Step One: Get an API Key.
First, go to https://api.data.gov/ and click on 'request a key' or 'API key signup'. Enter your name, email, and optional reason for using the API, then click 'signup' and check your email for the key.

## Step Two: Documentation.
Next, go to the OpenFEC API's documentation page to refer to. This is available at:
https://api.open.fec.gov/developers/. The FEC API is a powerful tool, and you will need to lookup the specific syntax for the data you are interested in, as well as the different URLs we will need.

## Step Three: Initialize API Variables and Import Libraries
Step three is to initialize our API variables, to make data search easier.

In [ ]:
# This first variable is the API key we obtained in step one. I have left this blank, please insert your key between the "".
key = ""

# Second variable is the base search URL.
base_url = "http://api.open.fec.gov/v1"

# Next we will import the libraries we will need
import requests #requests library, enables get requests
import json #data structure, similar to dictionaries
import pandas #we know this one!

## Step Four: Candidate Search
Step four is to query the API to find candidate variable numbers that are assigned by the FEC once a candidate raises or spends $5,000 or above. We will search for candidates whom ran for the House of Representatives from the state of Connecticut in 2018.

In [ ]:
# The first variable is URL extension needed for candidate searches.
search_url = base_url + "/candidates/search/"

# The next variable is our search parameters.
searchparams = {"api_key" : key,# Our API key
         "election_year" : 2018, # We are interested in the 2018 midterms for this exercise. 
         "office" : "H", # H = House of Representatives, see the documentation for other variables.
         "state" : "CT", # We are interested in Connecticut for this exercise.
         "sort" : "district", # We are sorting the list by district, but this can be any other variable.
         "per_page": "100",  # We are maximizing the number of hits we can get per page of data retrieval.
         "page" : '1'}

# Next, we perform the actual search.
candidate_search = requests.get(search_url, params=searchparams)

## Step Five: Convert Data
Next we will convert the data returned by our candidate into JSON data.

In [ ]:
# First we will convert data into text.
candidatetext = candidate_search.text

# Next we will convert text into JSON
candidatedata = json.loads(candidatetext)

# We can now find the keys, and then convert results into a pandas dataframe, then sort the dataframe by district.
candidatedata.keys()
type(candidatedata['results'])
candidatedf = pandas.DataFrame(candidatedata['results'])
candidatedf

## Step Six: Use Candidate ID to Retrieve Campaign Finance Data
We will now use the candidate ID of John B. Larson from the data frame above. Larson has represented Connecticut's first district since 1998 and won office again in the 2018 midterms.

In [ ]:
# We will now go back to the API documentation and create the new API URL.
finance_url = base_url + "/candidate/H8CT01046/totals"

# Now we create new search parameters.
financeparams = {"api_key" : key, # Our API Key
                 "full_election" : 1} # A boolean operator to retrieve data for full election cycles.

# Next we perform the actual query.
finance_search = requests.get(finance_url, params=financeparams)                

In [ ]:
# Convert our data into text.
financetext = finance_search.text

# Convert text into JSON.
financedata = json.loads(financetext)
financedata.keys()
financedf = pandas.DataFrame(financedata['results'])
financedf

## Step Seven: Visualize data
In the next step we will visualize the total campaign disbursments for all of Mr. Larson's campaigns.

In [ ]:
# First we import our visualization library.
import matplotlib.pyplot as plt

# Next we will group the data by the cycle year.
plt.bar(x=financedf['cycle'], height=financedf['disbursements'], align = 'center')
plt.rcParams["figure.figsize"] = [10,10]
plt.xticks(ticks=financedf['cycle'])
plt.xlabel('Election Cycle')
plt.ylabel('Total Disbursement ($)')
plt.show()

## Conclusion
The FEC API gives us the ability to look up campaign spending as well as many other variables. Representative John B. Larson, a Democrat from Connecticut's first district, spent nearly 2.5 million dollars in the 2010 election cycle, the peak of his campaign spending. 2010 spending may have been expenses necessary to fight off a Tea Party insurgent Republican or perhaps another campaign reason like the *Citizens United* ruling. Note that this spending is only from Representative Larson's campaign itself and does not include other spending like from Political Action Committees.